<a href="https://colab.research.google.com/github/dbenayoun/IASD/blob/main/importGolois.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
!unzip project2022.zip

--2024-05-06 18:21:22--  https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138884468 (132M) [application/zip]
Saving to: ‘project2022.zip’

project2022.zip     100%[===================>] 132.45M  20.2MB/s    in 7.5s    

2024-05-06 18:21:31 (17.8 MB/s) - ‘project2022.zip’ saved [138884468/138884468]

Archive:  project2022.zip
  inflating: Board.h                 
  inflating: Game.h                  
  inflating: Rzone.h                 
  inflating: compile.sh              
  inflating: compileMAC.sh           
  inflating: games.data              
  inflating: golois.cpp              
  inflating: golois.cpython-310-x86_64-linux-gnu.so  
  inflating: golois.cpython-37m-x86_64-linux-gnu.so  
  inflating: golois.cpython-38-x86_64-linux-gnu.so  
  inflat

In [4]:
!python golois.py

2024-05-06 18:28:33.777211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 18:28:33.777254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 18:28:33.778595: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 18:28:33.785871: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 18:28:34.761403: W tensorflow/comp

In [20]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import gc
import pandas as pd

In [6]:
import golois


In [7]:
planes = 31
moves = 361
N = 10000
epochs = 20
batch = 128
filters = 32

In [8]:
input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)

getValidation


In [24]:
input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
for i in range (5):
    ident = x
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = layers.add([x, ident])
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)
value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])



1. added a residual layer


In [25]:
#model.summary ()

model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

for i in range (1, epochs + 1):
    print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups, i * N)
    history = model.fit(input_data,
                        {'policy': policy, 'value': value},
                        epochs=1, batch_size=batch)
    if (i % 5 == 0):
        gc.collect ()
    if (i % 20 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("val =", val)
        model.save ('test.h5')

epoch 1
79/79 [==============================] - 3s 14ms/step - loss: 6.3621 - policy_loss: 5.6597 - value_loss: 0.6931 - policy_categorical_accuracy: 0.0104 - value_mse: 0.1215
epoch 2
79/79 [==============================] - 1s 18ms/step - loss: 5.9274 - policy_loss: 5.2250 - value_loss: 0.6931 - policy_categorical_accuracy: 0.0216 - value_mse: 0.1194
epoch 3
79/79 [==============================] - 1s 14ms/step - loss: 5.7393 - policy_loss: 5.0369 - value_loss: 0.6931 - policy_categorical_accuracy: 0.0260 - value_mse: 0.1204
epoch 4
79/79 [==============================] - 1s 14ms/step - loss: 5.5451 - policy_loss: 4.8427 - value_loss: 0.6931 - policy_categorical_accuracy: 0.0423 - value_mse: 0.1207
epoch 5
79/79 [==============================] - 1s 16ms/step - loss: 5.3037 - policy_loss: 4.6013 - value_loss: 0.6931 - policy_categorical_accuracy: 0.0736 - value_mse: 0.1195
epoch 6
79/79 [==============================] - 1s 16ms/step - loss: 5.0102 - policy_loss: 4.3078 - value_los

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
results = {'Metrics': ['loss', 'policy_loss', 'value_loss', 'policy_categorical_accuracy', 'value_mse'], 'Prev_val': prev_val, 'val': val}
pd.DataFrame(results)

,Metrics,Prev_val,val
0,loss,4.719369,4.334654
1,policy_loss,4.016899,3.632259
2,value_loss,0.692867,0.693021
3,policy_categorical_accuracy,0.156900,0.208300
4,value_mse,0.119907,0.119984


In [ ]:
prev_val = val.copy()